reuterのデータセットでclean labelと noisy labelでどれぐらい判別性能に差が出るのかを検証する(上限と下限づくりということ)

なぜreuter(RCV1-v2)?→ラベルが正しいので

In [ ]:
from multiprocessing import cpu_count
import numpy as np

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import rcv1

from cleanlab.noise_generation import generate_noise_matrix, generate_noise_matrix_from_trace, generate_noisy_labels
from cleanlab.classification import LearningWithNoisyLabels

# visualize
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%config InlineBackend.figure_formats = {'png', 'retina'}

seed=42

### dataの読み込み

In [ ]:
data=rcv1.fetch_rcv1()

In [ ]:
# データセットがマルチクラス判別なので、シングルクラスにまとめる
mask_col=np.array(list(map(lambda x:x.endswith('CAT'), data.target_names))) #カテゴリーに分けるのが良さそう
target_names=data.target_names[mask_col]
print('target names', target_names) #C→corporate inductrial, E→economics, G→goverment, M→わからない...Market?
mask_row=data.target[:,mask_col].toarray().sum(axis=1) == 1 #マルチクラスが割り当てられているサンプルは削除

y=data.target[mask_row][:,mask_col]
X=data.data[mask_row]
py=y.toarray().sum(axis=0).reshape(-1) #given labelの数
print('samples',X.shape[0],'category value counts',py)
y=np.array(y.argmax(axis=1)).reshape(-1) #one-hot to num

評価に関して、本当はcross validationをするべきなのかもしれない。
しかし今回は簡単のため1つのtestだけで評価する。

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=seed, shuffle=True)

### clean labelで学習
LogRegなのは比較的軽量っていう理由。非線形モデルとしてRandomForestsやExtraTreesも試したが、時間がかかる割にこのタスクにおける判別性能は微妙だった。

%%time
clf=LogisticRegression(solver='lbfgs',
                       multi_class='multinomial',
                       verbose=2,
                       random_state=seed
                      )

clf.fit(X_train,y_train)
print(clf.score(X_test, y_test)) #めっちゃあたる

In [ ]:
%%time
clf=LogisticRegression(multi_class='auto',
                       verbose=2,
                       random_state=seed
                      )

clf.fit(X_train,y_train)
print(clf.score(X_test, y_test)) #悪くないけどノイズの影響受けてる。比較的頑健な線形モデルでもこう

### noisy labelで学習

In [ ]:
# p(given=i | true=j) の行列を生成
noise_matrix=generate_noise_matrix_from_trace(4,3,
                                             min_trace_prob=0.6,
                                             frac_zero_noise_rates=0.5,
                                             py=py,
                                             seed=seed,
                                            )
print('p(given=i|true=j) =')
print(noise_matrix)
y_train_corrupted=generate_noisy_labels(y_train,noise_matrix)
n=(y_train_corrupted!=y_train).sum()
print('y_train_copputed contains', n ,'errors. error rate is', int(n/len(y_train) * 100),'%')

%%time
clf=LogisticRegression(solver='lbfgs',
                       multi_class='multinomial',
                       verbose=2,
                       random_state=seed
                      )

clf.fit(X_train,y_train_corrupted)
print(clf.score(X_test, y_test)) #悪くないけどノイズの影響受けてる。比較的頑健な線形モデルでもこう

In [ ]:
%%time
clf=LogisticRegression(multi_class='auto',
                       verbose=2,
                       random_state=seed
                      )

clf.fit(X_train,y_train_corrupted)
print(clf.score(X_test, y_test)) #悪くないけどノイズの影響受けてる。比較的頑健な線形モデルでもこう

### Confident Learningでnoisy labelを除外して学習

%%time
model=LogisticRegression(solver='lbfgs',
                       multi_class='multinomial',
                       verbose=2,
                       random_state=seed)

clf=LearningWithNoisyLabels(clf=model,
                            seed=seed,
                            n_jobs=cpu_count())
                            
clf.fit(X_train,y_train_corrupted)
print(clf.score(X_test, y_test)) #悪くないけどノイズの影響受けてる

In [ ]:
%%time
model=LogisticRegression(multi_class='auto',
                       verbose=2,
                       random_state=seed)

clf=LearningWithNoisyLabels(clf=model,
                            seed=seed,
                            n_jobs=cpu_count())
                            
clf.fit(X_train,y_train_corrupted)
print(clf.score(X_test, y_test)) #悪くないけどノイズの影響受けてる

### confident learningでどれぐらいのノイズが検出できているのか